In [122]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from pathlib import Path
import torch.nn as nn
import warnings
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from tqdm import tqdm
import time
import numpy as np
from PIL import Image
from collections import Counter
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [123]:
# Settings
train_dir = Path('./data/train')
val_dir   = Path('./data/test')
img_size = 48
batch_size = 64

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomAffine(degrees=0, shear=10, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # normalize to [-1, 1]
])

val_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(root=val_dir, transform=val_transforms)
print(f"Train images: {len(train_dataset)}")
print(f"Val images: {len(val_dataset)}")

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Class names
class_names = train_dataset.classes
print("Classes:", class_names)

Train images: 33595
Val images: 7178
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [124]:
import torch.nn as nn
import torch.nn.functional as F

class ResSim(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.15):
        super(ResSim, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)

        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 24, kernel_size=3, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.Conv2d(24, 24, kernel_size=3, padding=1),
            nn.BatchNorm2d(24)
        )
        self.shortcut1 = nn.Conv2d(3, 24, kernel_size=1)

        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(24, 48, kernel_size=3, padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, kernel_size=3, padding=1),
            nn.BatchNorm2d(48)
        )
        self.shortcut2 = nn.Conv2d(24, 48, kernel_size=1)

        self.pool = nn.MaxPool2d(2, 2)

        # Final FC layer stays at 64
        self.fc = nn.Sequential(
            nn.Linear(12 * 12 * 48, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x1 = self.block1(x)
        x = self.pool(F.relu(x1 + self.shortcut1(x)))
        x = self.dropout(x)

        x2 = self.block2(x)
        x = self.pool(F.relu(x2 + self.shortcut2(x)))
        x = self.dropout(x)

        x = x.view(x.size(0), -1)
        return self.fc(x)

In [125]:
model = ResSim(num_classes=7)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 481551


In [126]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResSim(num_classes=7).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [127]:
from sklearn.metrics import classification_report
import time
from tqdm import tqdm

def train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name='resnet_model.pth'):
    history = []  # collect metrics per epoch

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        start_time = time.time()

        # Training phase
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss /= total

        # Validation phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        y_true, y_pred = [], []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        val_acc = correct / total
        val_loss /= total

        # Classification report
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        val_f1_macro = report['macro avg']['f1-score']

        epoch_time = time.time() - start_time

        # Collect metrics
        epoch_data = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'val_f1_macro': val_f1_macro,
            'epoch_time_sec': epoch_time
        }

        # Add per-class metrics
        for cls in class_names:
            for metric in ['precision', 'recall', 'f1-score']:
                key = f'{cls}_{metric}'
                epoch_data[key] = report[cls][metric]

        history.append(epoch_data)

        # Console output
        print("\nValidation Report:")
        for cls in class_names:
            cls_metrics = report[cls]
            print(f"{cls}: Prec={cls_metrics['precision']:.3f} | Rec={cls_metrics['recall']:.3f} | F1={cls_metrics['f1-score']:.3f}")

        print(f"\nEpoch Summary: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Train Acc={train_acc:.3f} | Val Acc={val_acc:.3f}")
        print(f"Epoch time: {epoch_time:.2f} seconds")

    torch.save(model.state_dict(), f"results/{model_name}")
    print(f"Saved model to 'results/{model_name}'")

    return history

In [128]:
# metrics_history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15)
# df_metrics = pd.DataFrame(metrics_history)
# display(df_metrics.head())

In [ ]:
def build_model(dropout):
    return ResSim(num_classes=len(class_names), dropout_rate=dropout).to(device)

# Define settings to test
configs = [
    {'name': 'adam_lr3_dropout0_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0},
    {'name': 'adam_lr3_dropout15_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.15},
    {'name': 'adam_lr2_dropout15_bn', 'optimizer': 'adam', 'lr': 1e-2, 'dropout': 0.15},
    {'name': 'adam_lr3_dropout20_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.20},
    {'name': 'sgd_mom09_lr3_dropout15_bn', 'optimizer': 'sgd', 'lr': 1e-3, 'dropout': 0.15},
]


for config in configs:
    print(f"\nTraining config: {config['name']}")

    model = build_model(config['dropout'])
    if config['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    else:
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)

    criterion = nn.CrossEntropyLoss()
    history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name=config['name'])
    pd.DataFrame(history).to_csv(f'results/history_{config["name"]}.csv', index=False)


Training config: adam_lr3_dropout0_bn

Epoch 1/15


Training:  14%|█▍        | 76/525 [00:17<01:18,  5.72it/s]

# Small test

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from collections import defaultdict
import numpy as np
import random

# Transform (with normalization to [-1, 1] range)
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # For 3 channels, use 3 values if needed
])

# Load full dataset
full_train_dataset = datasets.ImageFolder('./data/train', transform=transform)

# Limit to 500 samples per class
class_counts = defaultdict(list)
for idx, (_, label) in enumerate(full_train_dataset):
    if len(class_counts[label]) < 500:
        class_counts[label].append(idx)

# Flatten selected indices
subset_indices = [idx for indices in class_counts.values() for idx in indices]

# Create subset dataset
small_train_dataset = Subset(full_train_dataset, subset_indices)
train_loader = DataLoader(small_train_dataset, batch_size=32, shuffle=True)

# Validation loader (no subsample here, or you can do similarly)
val_dataset = datasets.ImageFolder('./data/test', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
metrics_history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15)


Epoch 1/15


Validation: 100%|██████████| 225/225 [00:17<00:00, 12.95it/s]



Validation Report:
angry: Prec=0.271 | Rec=0.062 | F1=0.100
disgust: Prec=0.184 | Rec=0.568 | F1=0.278
fear: Prec=0.232 | Rec=0.132 | F1=0.168
happy: Prec=0.703 | Rec=0.623 | F1=0.660
neutral: Prec=0.305 | Rec=0.426 | F1=0.356
sad: Prec=0.251 | Rec=0.260 | F1=0.255
surprise: Prec=0.392 | Rec=0.686 | F1=0.499

Epoch Summary: Train Loss=1.4209 | Val Loss=1.6420 | Train Acc=0.425 | Val Acc=0.387
Epoch time: 42.38 seconds

Epoch 2/15


Validation: 100%|██████████| 225/225 [00:07<00:00, 30.92it/s]



Validation Report:
angry: Prec=0.203 | Rec=0.073 | F1=0.107
disgust: Prec=0.151 | Rec=0.604 | F1=0.241
fear: Prec=0.200 | Rec=0.118 | F1=0.149
happy: Prec=0.725 | Rec=0.557 | F1=0.630
neutral: Prec=0.295 | Rec=0.441 | F1=0.354
sad: Prec=0.261 | Rec=0.163 | F1=0.200
surprise: Prec=0.350 | Rec=0.758 | F1=0.479

Epoch Summary: Train Loss=1.4369 | Val Loss=1.6770 | Train Acc=0.414 | Val Acc=0.365
Epoch time: 21.61 seconds

Epoch 3/15


...


Validation Report:
angry: Prec=0.225 | Rec=0.154 | F1=0.183
disgust: Prec=0.125 | Rec=0.649 | F1=0.210
fear: Prec=0.181 | Rec=0.079 | F1=0.110
happy: Prec=0.764 | Rec=0.521 | F1=0.620
neutral: Prec=0.331 | Rec=0.347 | F1=0.339
sad: Prec=0.242 | Rec=0.267 | F1=0.254
surprise: Prec=0.364 | Rec=0.709 | F1=0.481

Epoch Summary: Train Loss=1.2935 | Val Loss=1.8403 | Train Acc=0.472 | Val Acc=0.359
Epoch time: 20.96 seconds

Epoch 14/15


Validation: 100%|██████████| 225/225 [00:08<00:00, 27.18it/s]



Validation Report:
angry: Prec=0.198 | Rec=0.156 | F1=0.174
disgust: Prec=0.103 | Rec=0.595 | F1=0.175
fear: Prec=0.202 | Rec=0.096 | F1=0.130
happy: Prec=0.878 | Rec=0.348 | F1=0.499
neutral: Prec=0.343 | Rec=0.289 | F1=0.314
sad: Prec=0.243 | Rec=0.326 | F1=0.278
surprise: Prec=0.321 | Rec=0.727 | F1=0.446

Epoch Summary: Train Loss=1.2997 | Val Loss=1.8798 | Train Acc=0.460 | Val Acc=0.320
Epoch time: 21.74 seconds

Epoch 15/15


Validation: 100%|██████████| 225/225 [00:07<00:00, 29.84it/s]


Validation Report:
angry: Prec=0.249 | Rec=0.106 | F1=0.149
disgust: Prec=0.132 | Rec=0.604 | F1=0.217
fear: Prec=0.200 | Rec=0.105 | F1=0.138
happy: Prec=0.765 | Rec=0.523 | F1=0.621
neutral: Prec=0.380 | Rec=0.200 | F1=0.262
sad: Prec=0.251 | Rec=0.369 | F1=0.299
surprise: Prec=0.308 | Rec=0.755 | F1=0.438

Epoch Summary: Train Loss=1.2884 | Val Loss=1.7998 | Train Acc=0.466 | Val Acc=0.353
Epoch time: 19.14 seconds
Saved model to 'resnet_model.pth'


In [ ]:
df_metrics = pd.DataFrame(metrics_history)
display(df_metrics.head())

,epoch,train_loss,val_loss,train_acc,val_acc,val_f1_macro,epoch_time_sec,angry_precision,angry_recall,angry_f1-score,...,happy_f1-score,neutral_precision,neutral_recall,neutral_f1-score,sad_precision,sad_recall,sad_f1-score,surprise_precision,surprise_recall,surprise_f1-score
0,1,1.420878,1.642038,0.424857,0.387434,0.330929,42.379916,0.270642,0.061587,0.100340,...,0.660490,0.305410,0.425791,0.355691,0.251163,0.259824,0.255420,0.392022,0.685921,0.498906
1,2,1.436899,1.676996,0.414286,0.365422,0.308582,21.605242,0.202899,0.073069,0.107444,...,0.630102,0.295492,0.441200,0.353936,0.260591,0.162791,0.200395,0.349612,0.758123,0.478542
2,3,1.389826,1.732358,0.436571,0.350515,0.299288,18.324173,0.201613,0.130480,0.158428,...,0.603286,0.321819,0.453366,0.376431,0.240059,0.130714,0.169263,0.333683,0.766546,0.464964
3,4,1.398585,1.696068,0.432857,0.381861,0.319046,18.263938,0.218421,0.086639,0.124066,...,0.658976,0.317190,0.377129,0.344572,0.239544,0.202085,0.219226,0.382103,0.703971,0.495343
4,5,1.392669,1.872429,0.434857,0.332405,0.295151,18.198030,0.191419,0.181628,0.186395,...,0.599720,0.326797,0.243309,0.278940,0.239681,0.265437,0.251903,0.399718,0.681107,0.503783


In [ ]:
train(model, train_loader, val_loader, num_epochs=15)


Epoch 1/15


Validation: 100%|██████████| 225/225 [00:07<00:00, 28.24it/s]
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisio


Validation Report:
angry: Prec=0.000 | Rec=0.000 | F1=0.000
disgust: Prec=0.000 | Rec=0.000 | F1=0.000
fear: Prec=0.294 | Rec=0.005 | F1=0.010
happy: Prec=0.516 | Rec=0.745 | F1=0.609
neutral: Prec=0.282 | Rec=0.294 | F1=0.288
sad: Prec=0.233 | Rec=0.262 | F1=0.247
surprise: Prec=0.326 | Rec=0.750 | F1=0.455
✅ Saved new best model!

Epoch Summary: Train Loss=1.8949 | Val Loss=1.7561 | Train Acc=0.234 | Val Acc=0.368
⏱️ Epoch time: 19.77 seconds

Epoch 2/15


Validation: 100%|██████████| 225/225 [00:07<00:00, 28.84it/s]
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisio


Validation Report:
angry: Prec=0.000 | Rec=0.000 | F1=0.000
disgust: Prec=0.000 | Rec=0.000 | F1=0.000
fear: Prec=0.143 | Rec=0.001 | F1=0.002
happy: Prec=0.622 | Rec=0.652 | F1=0.636
neutral: Prec=0.302 | Rec=0.303 | F1=0.303
sad: Prec=0.243 | Rec=0.329 | F1=0.279
surprise: Prec=0.284 | Rec=0.813 | F1=0.421
✅ Saved new best model!

Epoch Summary: Train Loss=1.8487 | Val Loss=1.7447 | Train Acc=0.251 | Val Acc=0.365
⏱️ Epoch time: 19.99 seconds

Epoch 3/15


...


Validation Report:
angry: Prec=0.190 | Rec=0.099 | F1=0.130
disgust: Prec=0.121 | Rec=0.649 | F1=0.204
fear: Prec=0.206 | Rec=0.130 | F1=0.159
happy: Prec=0.771 | Rec=0.551 | F1=0.642
neutral: Prec=0.277 | Rec=0.522 | F1=0.362
sad: Prec=0.217 | Rec=0.069 | F1=0.105
surprise: Prec=0.416 | Rec=0.722 | F1=0.527
✅ Saved new best model!

Epoch Summary: Train Loss=1.4808 | Val Loss=1.6571 | Train Acc=0.409 | Val Acc=0.363
⏱️ Epoch time: 1667.29 seconds

Epoch 14/15


Validation: 100%|██████████| 225/225 [17:44<00:00,  4.73s/it]  



Validation Report:
angry: Prec=0.185 | Rec=0.063 | F1=0.094
disgust: Prec=0.085 | Rec=0.685 | F1=0.151
fear: Prec=0.181 | Rec=0.091 | F1=0.121
happy: Prec=0.745 | Rec=0.564 | F1=0.642
neutral: Prec=0.325 | Rec=0.182 | F1=0.234
sad: Prec=0.256 | Rec=0.296 | F1=0.275
surprise: Prec=0.328 | Rec=0.779 | F1=0.461

Epoch Summary: Train Loss=1.4382 | Val Loss=1.6970 | Train Acc=0.426 | Val Acc=0.344
⏱️ Epoch time: 2987.17 seconds

Epoch 15/15


Validation: 100%|██████████| 225/225 [17:13<00:00,  4.59s/it] 


Validation Report:
angry: Prec=0.184 | Rec=0.057 | F1=0.088
disgust: Prec=0.111 | Rec=0.595 | F1=0.187
fear: Prec=0.215 | Rec=0.141 | F1=0.170
happy: Prec=0.748 | Rec=0.555 | F1=0.638
neutral: Prec=0.307 | Rec=0.430 | F1=0.358
sad: Prec=0.232 | Rec=0.162 | F1=0.191
surprise: Prec=0.364 | Rec=0.745 | F1=0.489

Epoch Summary: Train Loss=1.4497 | Val Loss=1.6597 | Train Acc=0.423 | Val Acc=0.362
⏱️ Epoch time: 2019.59 seconds


## Freeze Layers

In [ ]:
import torch.nn as nn
import torchvision.models as models

def build_light_resnet(num_classes):
    # Load pretrained resnet18
    base_model = models.resnet18(pretrained=True)

    # Freeze all layers
    for param in base_model.parameters():
        param.requires_grad = False

    # Replace the final classifier
    base_model.fc = nn.Sequential(
        nn.Linear(base_model.fc.in_features, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(64, num_classes)
    )

    return base_model

In [ ]:
model = build_light_resnet(num_classes=len(class_names))

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")

/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/yuyi/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:06<00:00, 7.14MB/s]

Total Parameters: 11209927
Trainable Parameters: 33415
